<a href="https://colab.research.google.com/github/grand-cat-unknown/Street_View_House_Numbers/blob/master/Srikanth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
##GET DATA
!wget -nc http://ufldl.stanford.edu/housenumbers/train.tar.gz 
!wget -nc http://ufldl.stanford.edu/housenumbers/test.tar.gz 
!wget -nc http://ufldl.stanford.edu/housenumbers/extra.tar.gz 
!git clone https://github.com/grand-cat-unknown/Street_View_House_Numbers SVHN

##ERROR IMAGES image = cv2.imread('train/30907.png')


File ‘train.tar.gz’ already there; not retrieving.



In [0]:
# from google.colab import files

# files.download('train.tar.gz')

In [0]:
##IMPORTS 
import os
import tarfile
import numpy as np
import h5py
import pickle

import pickle
import cv2
import json
import glob
import natsort  # natural sort (1, 2, 3, 4..., 100), instead of (1, 10, 100, 1000....)
from tqdm import tqdm

from SVHN.utils.hdf5datasetwriter import HDF5DatasetWriter
from SVHN.utils.aspect_aware_resize import AspectAwareResize
from SVHN.utils.extract_patch import ExtractPatch

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline



## IMPORT GITHUB

if 'train' not in os.listdir():
  my_tar = tarfile.open('train.tar.gz')
  my_tar.extractall('')
  my_tar.close()

if 'test' not in os.listdir():
  my_tar = tarfile.open('test.tar.gz')
  my_tar.extractall('')
  my_tar.close()

if 'extra' not in os.listdir():
  my_tar = tarfile.open('extra.tar.gz')
  my_tar.extractall('')
  my_tar.close()

In [0]:

def get_box_data(index, hdf5_data):
    """
    get `left, top, width, height` of each picture
    :param index:
    :param hdf5_data:
    :return:
    """
    meta_data = dict()
    meta_data['label'] = []
    meta_data['top'] = []
    meta_data['left'] = []
    meta_data['height'] = []
    meta_data['width'] = []

    def print_attrs(name, obj):
        vals = []
        if obj.shape[0] == 1:
            vals.append(obj[0][0])
        else:
            for k in range(obj.shape[0]):
                vals.append(int(hdf5_data[obj[k][0]][0][0]))
        meta_data[name] = vals

    box = hdf5_data['/digitStruct/bbox'][index]
    hdf5_data[box[0]].visititems(print_attrs)
    return meta_data

def get_name(index, hdf5_data):
    name = hdf5_data["/digitStruct/name"]
    print(''.join([chr(v[0]) for v in hdf5_data[name[index][0]].value]))



In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# f = h5py.File('test/digitStruct.mat')

In [0]:

#MAKE SIZE AS 500

if "metadata.pkl"  not in os.listdir('/content/drive/My Drive/SVHN/'):
  labels_file = "train/digitStruct.mat"
  f = h5py.File(labels_file)
  metadata = []
  for j in range(len(os.listdir('train/'))- 3):
    print(j)
    metadata.append(get_box_data(j, f))
  pickle.dump(metadata, open("/content/drive/My Drive/SVHN/metadata.pkl", "wb"))

if "metadata_test.pkl" not in os.listdir('/content/drive/My Drive/SVHN/'):
  labels_file = "test/digitStruct.mat"
  f = h5py.File(labels_file)
  metadata_test = []
  for j in range(len(os.listdir('test/'))- 3):
    print(j)
    metadata_test.append(get_box_data(j, f))
  pickle.dump(metadata_test, open("/content/drive/My Drive/SVHN/metadata_test.pkl", "wb"))


if "metadata_extra.pkl" not in os.listdir('/content/drive/My Drive/SVHN/'):
  labels_file = "extra/digitStruct.mat"
  f = h5py.File(labels_file)
  metadata_extra = []
  for j in range(30000):
    print(j)
    metadata_extra.append(get_box_data(j, f))
  pickle.dump(metadata_extra, open("/content/drive/My Drive/SVHN/metadata_extra.pkl", "wb"))



In [0]:
metadata = pickle.load(open('/content/drive/My Drive/SVHN/metadata.pkl','rb'))
metadata_test = pickle.load(open('/content/drive/My Drive/SVHN/metadata_test.pkl','rb'))
metadata_extra = pickle.load(open('/content/drive/My Drive/SVHN/metadata_extra.pkl','rb'))

# metadata[20]

In [0]:
##ONLY A FEW EXIST
for j, i in enumerate(range(len(os.listdir('train/'))- 3)):
    t = len(metadata[i]["label"])
    if t >= 5:
        print(j, t)

In [0]:
final_labels = []

metadata = pickle.load(open("/content/drive/My Drive/SVHN/metadata.pkl", "rb"))
for i in range(len(os.listdir('train/'))- 5):
    t = metadata[i]["label"]
    temp_label = [int(x) for x in t]
    dt = len(temp_label) - 5
    if dt < 0:
        for ii in range(abs(dt)):
            temp_label.append(11)
    if dt > 0:
        del temp_label[-1]
    tt = len(t) if len(t)< 6 else 5
    temp_label.append(tt)
    final_labels.append(temp_label)
y = np.array(final_labels)
pickle.dump(y, open("/content/drive/My Drive/SVHN/svhn_labels.pkl", "wb"))


metadata_test = pickle.load(open("/content/drive/My Drive/SVHN/metadata_test.pkl", "rb"))

final_labels_test = []
for i in range(len(os.listdir('test/'))- 5):
    t = metadata_test[i]["label"]
    temp_label = [int(x) for x in t]
    dt = len(temp_label) - 5
    if dt < 0:
        for ii in range(abs(dt)):
            temp_label.append(11)
    if dt > 0:
        del temp_label[-1]
    tt = len(t) if len(t)< 6 else 5
    temp_label.append(tt)
    final_labels_test.append(temp_label)

y_test = np.array(final_labels_test)
pickle.dump(y_test, open("/content/drive/My Drive/SVHN/svhn_labels_test.pkl", "wb"))


metadata_extra = pickle.load(open("/content/drive/My Drive/SVHN/metadata_extra.pkl", "rb"))

final_labels_extra = []
for i in range(30000):
    t = metadata_extra[i]["label"]
    temp_label = [int(x) for x in t]
    dt = len(temp_label) - 5
    if dt < 0:
        for ii in range(abs(dt)):
            temp_label.append(11)
    if dt > 0:
        del temp_label[-1]
    tt = len(t) if len(t)< 6 else 5
    temp_label.append(tt)
    final_labels_extra.append(temp_label)

y_extra = np.array(final_labels_extra)
pickle.dump(y_extra, open("/content/drive/My Drive/SVHN/svhn_labels_extra.pkl", "wb"))

In [0]:
len(metadata_extra)

In [0]:
image_paths = natsort.natsorted(glob.glob("train/*.png"))
test_paths = natsort.natsorted(glob.glob("test/*.png"))
extra_paths = natsort.natsorted(glob.glob("extra/*.png"))

metadata = pickle.load(open("/content/drive/My Drive/SVHN/metadata.pkl", "rb"))
labels = pickle.load(open("/content/drive/My Drive/SVHN/svhn_labels.pkl", "rb"))
metadata_test = pickle.load(open("/content/drive/My Drive/SVHN/metadata_test.pkl", "rb"))
labels_test = pickle.load(open("/content/drive/My Drive/SVHN/svhn_labels_test.pkl", "rb"))

metadata_extra = pickle.load(open("/content/drive/My Drive/SVHN/metadata_extra.pkl", "rb"))
labels_extra = pickle.load(open("/content/drive/My Drive/SVHN/svhn_labels_extra.pkl", "rb"))


In [0]:
# sl = glob.glob("test/*.png")[:]
# natsort.natsorted(sl)
labels_test[0]

In [0]:
# natsort.natsorted(glob.glob("test/*.png"))

In [0]:
def crop_bbox(img, metadata):
    shape = img.shape[:2]
    # print(shape)
    x = int(min(metadata["left"]))
    y = int(min(metadata["top"]))
    w = int(metadata["left"][-1] + metadata["width"][-1])
    h = int(max(metadata["top"]) + max(metadata["height"]))
    # print (x,y,w,h)
    dW = int(0.15*(w-x))
    dH = int(0.15*(h-y))
    
    x = 0 if (x - dW) < 0 else (x - dW)
    y = 0 if (y - dH) < 0 else (y - dH)
    w = shape[1] if (w + dW) > shape[1] else (w + dW)
    h = shape[0] if (h + dH) > shape[0] else (h + dH)
    # print (x,y,w,h)
    cropped = img[y:h, x:w]

    if cropped.shape[0] == 0 or cropped.shape[1] ==0:
      cropped = img
    # print(cropped.shape)
    return cv2.resize(cropped, (64, 64))

In [0]:
plt.figure(figsize=(20, 8))
import random

# x = random.randrange(7000)
x= 0
print(test_paths)

for idx, i in enumerate(range(x, x+32)):
    img = cv2.imread(test_paths[i])
    md = metadata_test[i]
    
    bb = crop_bbox(img, md)
    
    plt.subplot(4, 8, idx+1)
    plt.title(labels_test[i])
    plt.imshow(cv2.cvtColor(bb, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    
plt.show()

In [0]:
Ptr, ytr, metatr = image_paths[:-2500], labels[:-2500], metadata[:-2500]
Pval, yval, metaval = image_paths[-2500:], labels[-2500:], metadata[-2500:]

In [0]:
os.listdir("/content/drive/My Drive/SVHN/classify")


In [0]:
R, G, B = [], [], []


if 'train.hdf5' not in os.listdir("/content/drive/My Drive/SVHN/classify"):

  !rm classify/train.hdf5
  !rm classify/val.hdf5
  !rm classify/test.hdf5
  !mkdir /content/drive/My\ Drive/SVHN/classify
  !mkdir classify
  datasets = [("train", Ptr, ytr, metatr, "classify/train.hdf5"),
            ("val", Pval, yval, metaval, "classify/val.hdf5"),
              ("test", test_paths, labels_test, metadata_test, "classify/test.hdf5")]


  for (dtype, paths, labels, metas, outputs) in datasets:
      writer = HDF5DatasetWriter((len(paths), 64, 64, 3), outputs)
      # print(dtype)
      t = tqdm(zip(paths, labels, metas))
      for (path, label, meta) in t:
          # print(path)
          image = cv2.imread(path)
          image = crop_bbox(image, meta)
          
          if dtype == "train":
              (b, g, r) = cv2.mean(image)[:3]
              R.append(r)
              G.append(g)
              B.append(b)
              
          writer.add([image], [label])
      writer.close()
  !cp classify/train.hdf5 /content/drive/My\ Drive/SVHN/classify/train.hdf5
  print('copied train')
  !cp classify/test.hdf5 /content/drive/My\ Drive/SVHN/classify/test.hdf5
  print('copied test')
  !cp classify/val.hdf5 /content/drive/My\ Drive/SVHN/classify/val.hdf5
  print('copied val')

  

if 'extra.hdf5' not in os.listdir("/content/drive/My Drive/SVHN/classify"):
  !rm classify/extra.hdf5
  writer = HDF5DatasetWriter((30000, 64, 64, 3), "classify/extra.hdf5")
  t = tqdm(zip(extra_paths, labels_extra, metadata_extra))
  for (path, label, meta) in t:
      image = cv2.imread(path)
      image = crop_bbox(image, meta)

      (b, g, r) = cv2.mean(image)[:3]
      R.append(r)
      G.append(g)
      B.append(b)

      writer.add([image], [label])
  writer.close()
  !cp classify/extra.hdf5 /content/drive/My\ Drive/SVHN/classify/extra.hdf5



# M = {"R": np.mean(R), "G": np.mean(G), "B": np.mean(B)}
# pickle.dump(M, open("/content/drive/My Drive/SVHN/rgb_means.pkl", "wb"))

In [0]:
X = pickle.load(open("/content/drive/My Drive/SVHN/rgb_means.pkl", "rb"))
M = {"R": np.mean(R)+ X['R'], "G": np.mean(G) + X['G'] , "B": np.mean(B) + X['B']}
pickle.dump(M, open("/content/drive/My Drive/SVHN/rgb_means.pkl", "wb"))

In [0]:
%tensorflow_version 1.x 
import h5py
from keras.utils.np_utils import to_categorical

In [0]:
%tensorflow_version 1.x 
from SVHN.model import SVHNNet
import pickle
import numpy as np
import cv2
from SVHN.utils.extract_patch import ExtractPatch
from SVHN.utils.mean_subtract import MeanSubtract
from SVHN.utils.generator import Generator

from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
width = 54
height = 54
depth = 3
classes = 11
BS = 64
epochs = 50

In [0]:
model = SVHNNet.build(height, width, depth, classes)
model.summary()

In [0]:
means = pickle.load(open("/content/drive/My Drive/SVHN/rgb_means.pkl", "rb"))

ep = ExtractPatch(width, height)
ms = MeanSubtract(means["R"], means["G"], means["B"])
preprocessors = [ep, ms]

def exp_decay(epoch):
    initial_lrate = 0.001
    k = 0.1
    lrate = initial_lrate * np.exp(-k*epoch)
    
    return lrate

In [0]:
# model = load_model("/content/drive/My Drive/SVHN/svhn.h5")
opt = Adam(lr=0.0005)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
callbacks = [TensorBoard("logs", batch_size=BS, write_images=True),
            LearningRateScheduler(exp_decay)]

In [0]:
# import h5py
# filename = '/content/drive/My Drive/SVHN/classify/val.hdf5'

# with h5py.File(filename, 'r') as f:
#     # List all groups
#     print("Keys: %s" % f.keys())
#     a_group_key = list(f.keys())[1]

#     # Get the data
#     data = list(f[a_group_key])
#     print(data[0])

In [0]:
# !mkdir classify
# if 'train' not in os.listdir('classify/'):
#   !cp  /content/drive/My\ Drive/SVHN/classify/train.hdf5 classify/train.hdf5
#   !cp  /content/drive/My\ Drive/SVHN/classify/test.hdf5 classify/test.hdf5
#   !cp  /content/drive/My\ Drive/SVHN/classify/val.hdf5 classify/val.hdf5
extra_gen = Generator("/content/drive/My Drive/SVHN/classify/extra.hdf5", BS, epochs, preprocessors)
train_gen = Generator("/content/drive/My Drive/SVHN/classify/train.hdf5", BS, epochs, preprocessors)
val_gen = Generator("/content/drive/My Drive/SVHN/classify/val.hdf5", BS, epochs, preprocessors)

# callbacks, add in earlystopping
callbacks = [TensorBoard("logs", batch_size=BS, write_images=True),
            LearningRateScheduler(exp_decay),
            EarlyStopping(monitor="val_loss", min_delta=0.05, patience=20)]

In [0]:
H = model.fit_generator(extra_gen.generate(), steps_per_epoch=extra_gen.n_img//BS,
                         epochs=10, callbacks=callbacks)

In [0]:
model.save("/content/drive/My Drive/SVHN/svhn.h5")
model.save_weights("/content/drive/My Drive/SVHN/weights_model.h5")
H = model.fit_generator(train_gen.generate(), validation_data=val_gen.generate(), steps_per_epoch=train_gen.n_img//BS,
                        validation_steps=val_gen.n_img//BS, epochs=10, callbacks=callbacks)

In [0]:
model.save("/content/drive/My Drive/SVHN/svhn.h5")
model.save_weights("/content/drive/My Drive/SVHN/weights_model.h5")
H = model.fit_generator(train_gen.generate(), validation_data=val_gen.generate(), steps_per_epoch=train_gen.n_img//BS,
                        validation_steps=val_gen.n_img//BS, epochs=30, callbacks=callbacks)

In [0]:
# from google.colab import files
# files.download('svhn.h5') 

In [0]:
%tensorflow_version 1.x
from keras.models import load_model

import pickle
import cv2
import h5py
import glob
import numpy as np
import natsort

from SVHN.utils.generator import Generator
from SVHN.utils.simple_resize import SimpleResize
from SVHN.utils.mean_subtract import MeanSubtract

from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:

# model = load_model("svhn.h5")

# model= load_model("/content/drive/My Drive/SVHN/svhn.h5")
# model.load_weights("/content/drive/My Drive/SVHN/weights_model.h5")


with h5py.File("/content/drive/My Drive/SVHN/classify/test.hdf5", "r") as f:
    labels = f["labels"][()]
# print((labels)[-10:])
means = pickle.load(open("/content/drive/My Drive/SVHN/rgb_means.pkl", "rb"))
sr = SimpleResize(54, 54)
ms = MeanSubtract(means["R"], means["G"], means["B"])
preprocessors = [sr, ms]
gen = Generator("/content/drive/My Drive/SVHN/classify/test.hdf5", BS, 1, preprocessors)

In [0]:
predictions = model.predict_generator(gen.generate(), steps=(gen.n_img//BS)+1) #steps+1 because not enough steps to loop all data
gen.close()

In [0]:
np.argmax(predictions[0], axis=-1)


In [0]:
accuracy = 0.0
print("Accuracies for every digits:")
for i in range(6):
    pred = predictions[i]
    pred = np.argmax(pred, axis=-1)
    label = labels[:, i]
    
    acc = accuracy_score(label, pred)
    
    if i != 5:
        print("Digit {}: {:.2f}".format(i+1, acc))
    else:
        print("Sequence length: {:.2f}\n".format(acc))
        
    accuracy += acc
        
print("Overall accuracy: {:.2f}".format(accuracy/6))

In [0]:
%matplotlib inline

test_paths = natsort.natsorted(glob.glob("test/*.png"))

idxs = np.random.choice(len(test_paths), size=32)

plt.figure(figsize=(20, 8))
for n, i in enumerate(idxs):
    img = cv2.imread(test_paths[i])
    
    pred = []
    for p in predictions:
        temp = np.argmax(p, axis=-1)
        pred.append(temp[i])
        
    plt.subplot(4, 8, n+1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title(pred)
    
plt.show()